# Clothing Classification 
**Purpose**:  
Build and train a neural network to classify images of clothing. This model is trained on 60,000 images that include 10 types of articles of clothing.  

Project based on [TensorFlow's classification example](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l03c01_classifying_images_of_clothing.ipynb#scrollTo=DvYmmrpIy6Y1)

In [ ]:
# install and import dependencies

!pip install -U tensorflow_datasets

In [ ]:
import tensorflow as tf

# import tensorflow datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

**Import the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset**  
This dataset contains 70,000 grayscale images in ten categories, each in grayscale 28 x 28 pixel images.

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

This project uses the Fashion MNIST as a replacement for the classic MNIST dataset for variety and a slightly more challenging problem than the classic. The dataset is relatively small and is used to verify that the algorithm works as expected.


Link to [Datasets API](https://www.tensorflow.org/datasets)




In [ ]:
# import the Fashion MNIST dataset using the Datasets API
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
# store the class names for the image labels
class_names = metadata.features['label'].names
print("Class names:  {}".format(class_names))

**Explore and preprocess the data**  


In [ ]:
# find the number of training and testing images
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

In [ ]:
# normalize the values of the pixels from the range [0,255] to [0,1]

# create a normalization function and apply it to the datasets

def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

# cache the datasets to keep them in memory after first load from disk
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

In [ ]:
# test plot for an image

# take one image and remove the color dimension by reshaping the array
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))

# plot the image
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# display the first 25 images from the training set, display the class name, 
# and verify the data is in the correct format

plt.figure(figsize=(10,10))
for i, (image,label) in enumerate(test_dataset.take(25)):
  image = image.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap=plt.cm.binary)
  plt.xlabel(class_names[label])
plt.show()

**Build the model**  

Configure the layers and compile the model.  

Layers:
- **input - Flatten**: transforms the images from a 2D array of 28x28px to 1D of 784px. Reformats data.
- **hidden - Dense**: densely connected layer of 128 neurons. Each neuron takes input from all 784 nodes in the previous layer, weighting that input according to learned hidden parameters and outputs a single value to the next layer
- **output - Dense**: a 128-neuron followed by 10-node *softmax* layer, with each node representing a class of clothing. Takes input from the 128 nodes in the layer before it and outputs a value in the range [0,1] that represents the probability of an image belonging to the specified class

In [ ]:
# setup the layers

model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28,28,1)),
      tf.keras.layers.Dense(128, activation=tf.nn.relu),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
# compile the model

# loss function: SparseCategoricalCrossentropy
# optimizer: adam
# metrics: accuracy

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

**Train the model**  
1. Define the iteration behavior
- Repeat until specified epochs
- Shuffle the order of examples
- Use batches of 32 variables
2. Train using the `model.fit` method
- Feed the model the training data
- Model learning go brr
- Epochs limit training to 5 full iterations

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

**Evaluate accuracy**  
Accuracy on test_dataset is lower than accuracy on training, but this is normal! We can expect performance to go down.

In [ ]:
# compare how the model performs on the test dataset

test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

**Make predictions**  
Make some predictions about images

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
predictions.shape

In [ ]:
# full probability info
predictions[0]

# see which label has the highest confidence value
np.argmax(predictions[0])

In [ ]:
test_labels[0]

In [ ]:
# look at full set of class predictions
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)
  

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])   
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0,1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')
  

In [ ]:
# look at the 0th image, predictions, and prediction array

i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# use th trained model to make a prediction about a single image
img = test_images[0]
print(img.shape)

# add image to a batch where it's the only member since tf.keras models are
# optimized to make predictions on a batch/collection of examples at once
img = np.array([img])
print(img.shape)

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

In [ ]:
np.argmax(predictions_single[0])